In [ ]:
코드는 기존 구조 그대로 유지하면서 종료 키워드(끝, 종료, 그만, 안녕)를 감지하면 상담을 종료합니다.

import speech_recognition as sr
import pyttsx3
import random
import time

# -------------------------
# 1️⃣ 감정 분석 모듈 (예시)
# -------------------------
def analyze_emotion(text):
    if any(word in text for word in ["화나", "짜증", "분노"]):
        return "angry"
    elif any(word in text for word in ["슬퍼", "우울", "힘들"]):
        return "sad"
    elif any(word in text for word in ["좋아", "행복", "기뻐"]):
        return "happy"
    elif any(word in text for word in ["놀랐", "충격", "헉"]):
        return "surprised"
    else:
        return "neutral"

# -------------------------
# 2️⃣ 시나리오 매핑
# -------------------------
scenarios = {
    "happy": [
        "좋아요! 오늘 기분이 좋다니 다행이네요.",
        "즐거운 일이 있었나 봐요! 계속 행복하세요."
    ],
    "sad": [
        "힘든 일이 있었군요. 괜찮아요, 조금 쉬어도 돼요.",
        "슬픈 기분이 느껴져요. 제가 옆에서 들어줄게요."
    ],
    "angry": [
        "화가 나셨군요. 깊게 숨을 쉬고 차분히 생각해볼까요?",
        "속상하셨겠어요. 잠시 진정한 후 이야기해요."
    ],
    "surprised": [
        "정말 놀라셨군요! 대단한 일이네요.",
        "예상치 못한 일이 있었나 봐요!"
    ],
    "neutral": [
        "그렇군요. 더 이야기해 볼까요?",
        "알겠습니다. 계속 말씀해주세요."
    ]
}

# -------------------------
# 3️⃣ TTS 초기화
# -------------------------
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # 말 속도
engine.setProperty('volume', 1.0)  # 볼륨 0.0~1.0

# -------------------------
# 4️⃣ STT 초기화
# -------------------------
recognizer = sr.Recognizer()
microphone = sr.Microphone()

# -------------------------
import os
import queue
import sounddevice as sd
import vosk
import json
import pyttsx3
import random
import time

# -------------------------


# -------------------------
# 4️⃣ 상담 종료 키워드
# -------------------------
exit_keywords = ["끝", "종료", "그만", "안녕"]

# -------------------------
# 5️⃣ Vosk STT 초기화
# -------------------------
model_path = "vosk-model-small-ko-0.22"  # 한국어 모델 경로
if not os.path.exists(model_path):
    print("모델이 없습니다. https://alphacephei.com/vosk/models 에서 다운로드 후 압축 해제하세요.")
    exit(1)

model = vosk.Model(model_path)
q = queue.Queue()

# -------------------------
# USB 마이크 장치 확인 후 device_index 설정
# -------------------------
# 터미널에서 확인: arecord -l
MIC_DEVICE_INDEX = 1  # USB 마이크 장치 index (확인 후 변경)
SPEAKER_DEVICE_INDEX = None  # 기본 USB 스피커 사용

def audio_callback(indata, frames, time_, status):
    if status:
        print(status)
    q.put(bytes(indata))

# -------------------------
# 6️⃣ 실시간 상담 루프
# -------------------------
engine.say("안녕하세요. 상담을 시작합니다. 말씀해주세요.")
engine.runAndWait()

try:
    with sd.RawInputStream(samplerate=16000, blocksize=8000, dtype='int16',
                           channels=1, callback=audio_callback,
                           device=MIC_DEVICE_INDEX):
        rec = vosk.KaldiRecognizer(model, 16000)
        print("=== 상담 시스템 시작 (USB 마이크 + 스피커) ===")

        while True:
            data = q.get()
            if rec.AcceptWaveform(data):
                result = rec.Result()
                result_dict = json.loads(result)
                user_text = result_dict.get("text", "")
                if not user_text:
                    continue

                print("사용자 입력:", user_text)

                # 상담 종료 키워드 감지
                if any(word in user_text for word in exit_keywords):
                    print("상담을 종료합니다. 안녕히 가세요!")
                    engine.say("상담을 종료합니다. 안녕히 가세요!")
                    engine.runAndWait()
                    break

                # 감정 분석
                emotion = analyze_emotion(user_text)
                print("감정 감지:", emotion)

                # 시나리오 매핑
                reply_text = random.choice(scenarios.get(emotion, scenarios["neutral"]))

                # 텍스트 키워드 기반 세부 시나리오
                if "친구" in user_text and emotion == "angry":
                    reply_text = "친구와 싸우셨군요. 화가 많이 나셨겠어요. 잠시 진정한 후 이야기를 해볼까요?"

                print("응답 시나리오:", reply_text)

                # TTS 출력
                engine.say(reply_text)
                engine.runAndWait()

except KeyboardInterrupt:
    print("사용자에 의해 종료되었습니다.")
except Exception as e:
    print("오류 발생:", e)

In [ ]:
# USB 마이크 + USB 스피커 기준 → MIC_DEVICE_INDEX 설정 필요

#오프라인 STT(Vosk) → 인터넷 필요 없음

#TTS(espeak) → pyttsx3로 CPU 가볍게 처리

#상담 종료 키워드 감지 → 종료 시 TTS 안내 후 루프 종료

#감정 분석 모듈 교체 가능 → analyze_emotion() 부분

In [ ]:
한국어 음성 인식 모델 다운로드 안내

Vosk 모델 다운로드 링크  (한국어 모델 다운로드: Vosk Models
 → vosk-model-small-ko-0.22)

모델명: vosk-model-small-ko-0.22

USB 마이크 장치 확인 방법

arecord -l 명령어로 마이크 리스트 확인

출력되는 리스트에서 원하는 마이크의 장치 번호(MIC_DEVICE_INDEX) 확인

USB 스피커 출력 설정 방법

기본 출력으로 자동 선택됨

필요 시 alsamixer 명령어로 볼륨 및 출력 장치 확인 및 조절 가능